# Мэтчинг товаров для маркетплейса

## Открытие файлов с данными и их изучение

Маркетплейс поставил перед нами задачу мэтчинга: необходимо разработать алгоритм, который для каждого товара из датасета `validation.csv` предложит 5 наиболее похожих товаров из `base.csv`. Оценивать качество алгоритма нужно оценивать по метрике accuracy@5 по ответам из датасета `validation_answer.csv`.  
  
Нам доступно 4 датасета с данными:
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id и вектор признаков размерностью 72
- `train.csv` - датасет с данными для обучения модели. У каждого товара есть уникальный id, вектор признаков размерностью 72 и id товара из датасета `base.csv`, который максимально похож на него по мнению экспертов
- `validation.csv` - датасет с набором товаров, для которых надо найти наиболее близкие товары из `base.csv`. У каждого товара есть уникальный id и набор признаков размерностью 72
- `validation_answer.csv` - правильные ответы к предыдущему файлу

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit, cross_val_score
!pip install faiss-gpu
import faiss
import requests
from zipfile import ZipFile
import urllib
import json
from urllib.parse import urlencode
import time
from google.colab import files
!pip  install catboost
from catboost import CatBoostClassifier, Pool
import os

In [2]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/BBEphK0EHSJ5Jw'

# Формируем ссылку с яндекс диском для загрузки
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('data.zip', 'wb') as f:
    f.write(download_response.content)

# Сохраняем zip file, в котором хранятся данные
zip_file = ZipFile('data.zip')

In [3]:
train = pd.read_csv(zip_file.open('train.csv'), index_col=0) # Обучающий датасет
base = pd.read_csv(zip_file.open('base.csv'), index_col=0) # Набор товаров
validation = pd.read_csv(zip_file.open('validation.csv'), index_col=0) # Товары, для которых ищем наиболее похожие
answers = pd.read_csv(zip_file.open('validation_answer.csv'), index_col=0) # Ответы к предыдущему сету товаров

In [4]:
# Удалим ненужные файлы для очистки памяти
del zip_file, download_response

delete_filepath = '/content/data.zip'

open(delete_filepath, 'w').close()
os.remove(delete_filepath)

In [5]:
def data_overview(data, corr=False):

    # Настроим вывод датасетов, чтобы показывались все столбцы
    pd.set_option('display.max_columns', None)

    # Вывод первых строчек датасета для лучшего понимания структуры
    display(data.head())

    # Выведем размер датасетае
    print(f'Размер датасета: {data.shape}')

    # Проверим датасет на наличие пропусков
    nulls = pd.DataFrame(data.isna().sum(), columns=['NA_count'])
    if sum(nulls['NA_count'] == 0):
        print('В датасете нет пропусков')
    else:
        display(nulls.query('NA_count != 0'))

    # Выведем информацию о распределении признаков
    display(data.describe())

    # Посмотрим на корреляцию между признаками
    if corr:
        display(data.corr())

In [6]:
# Обучающий датасет 'train'
data_overview(train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,-88.08269,-52.695540,-27.692442,4.872923,198.348000,-30.075249,-3.803569,-135.810610,-161.84137,-78.512180,-94.42894,898.436927,-70.140520,78.420360,108.032776,813.770071,-74.790880,12.610422,-183.821840,149.71584,-27.042316,-133.21217,106.420746,-303.939233,48.73079,58.185707,24.250950,-151.22410,-104.282265,-34.492810,-12.587054,2.622891,-120.969920,149.99164,-31.948470,82.314430,-115.830470,-243.30939,6.622036,-132.197660,68.71256,-38.806824,62.937435,-132.65445,89.189026,87.039780,-121.27988,-6.986934,-206.513820,29.485587,-77.029590,-132.38617,-105.427820,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,-177.60580,-84.995514,42.810810,-57.256332,96.792534,-19.261467,0.739535,50.619213,-155.26703,-78.659430,-92.76149,353.157741,-34.744545,82.487110,-28.450592,813.770071,-137.529630,26.595627,-136.783450,153.35791,48.810093,-115.92215,87.464220,-222.286354,25.12415,91.887140,-30.636870,-136.59314,-140.500120,-43.449757,-7.226884,8.265747,-117.915470,149.15090,-18.751057,95.315384,-60.093273,-83.82058,37.449867,-23.298859,74.06108,-7.139753,75.862400,-112.04511,82.857730,54.067215,-134.00539,-26.142574,-214.632110,-457.848461,21.459618,-137.41136,-40.812233,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,-86.34656,-35.666546,16.395317,-80.802850,137.908650,-23.532760,-47.256584,-16.650242,-194.50568,-78.372925,-69.32448,1507.231274,-52.500970,-34.165775,52.958652,813.770071,-18.021725,20.951107,-50.321780,158.76062,0.178065,-183.06967,99.053570,-1018.469545,-51.80112,97.766770,-10.865850,-144.42316,-133.819490,-78.902300,-17.200352,4.467452,-63.970737,154.63953,-30.211614,48.527400,-122.406640,-112.71362,53.461838,-31.117260,107.84151,16.482935,77.934480,-95.61873,91.460075,63.119510,-126.93925,8.066627,-195.677670,-163.120000,-72.830000,-139.22307,-52.031662,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,-146.51707,-159.469850,-13.844755,-6.113928,118.939255,-44.585907,9.559358,14.435648,-156.90683,-78.789320,-78.73709,1507.231274,19.957405,34.834290,-8.820732,813.770071,-125.606800,17.584084,-58.452904,141.28180,-54.959310,-136.98854,63.880493,-1018.469545,89.22893,65.919960,-24.078644,-152.33410,-91.199380,-28.225390,-4.767386,0.158236,-129.128660,122.95837,-30.800995,123.623400,-37.540867,-72.13980,71.240990,-168.115590,118.23645,-18.065195,37.255720,-137.69104,87.500770,62.437290,-131.26064,35.692660,-86.038830,-379.339090,-153.465770,-131.19829,-61.567047,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,-107.15302,-8.800034,-50.977800,29.457338,143.389310,5.614824,-45.274760,9.643625,-77.55463,-79.066610,-77.92646,1507.231274,16.612400,116.284290,33.754898,813.770071,-105.765335,6.523008,-19.812988,157.69392,-20.604088,-146.59128,78.849570,-780.449185,87.56077,73.036660,16.891030,-144.65790,-116.122150,-19.353254,-7.709266,-5.394988,-140.252120,193.18497,-53.147078,79.869446,-151.131350,-45.05616,79.796234,46.763016,47.68181,-

Размер датасета: (100000, 73)
В датасете нет пропусков


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.328679,7.664345,-43.667046,-146.118630,111.770592,-73.178792,-440.615118,21.231588,123.067891,126.120201,-141.168584,-69.228444,-42.559541,6.506719,149.077582,-23.942253,-2.219093,-12.964439,-138.714610,-79.159795,-97.474574,1297.269578,-3.600685,55.908014,23.470181,742.788817,-64.970553,16.368072,-86.201170,152.098083,-13.659323,-144.506022,89.747894,-587.551768,36.468210,73.109595,-13.866967,-146.487267,-118.336458,-41.131969,-9.261504,4.139978,-104.928806,122.240784,-40.320211,59.051981,-84.054387,-140.787854,46.137851,-34.136875,29.462927,-22.343451,67.581086,-120.052756,88.193676,69.538752,-132.336967,14.452681,-120.301843,-315.282239,-66.378660,-130.672183,-81.145553,36.898233,-152.223082,14.001695,67.799560,23.029277,73.412076,115.189717,-709.761548,-48.505704
std,25.803845,4.955651,39.111064,20.434841,47.700958,28.718853,279.017550,65.985214,6.492081,64.912752,41.607785,51.258647,40.819544,37.593800,34.167843,13.024184,20.978943,55.210920,48.737245,0.775560,28.382144,375.123794,50.063680,35.617513,40.802653,191.052074,63.160650,4.828575,49.511818,10.392531,29.191689,35.816357,28.312124,273.294442,39.756311,20.008170,25.824911,10.383975,13.578397,19.868303,8.295381,5.922103,56.519443,48.109909,14.739497,31.088173,66.055063,59.851232,46.328286,59.325454,60.934591,54.991787,12.988132,55.920785,4.824051,12.339684,7.952113,49.234745,55.730177,210.657513,64.913422,9.408099,30.675871,25.357686,41.020285,99.079597,1.838012,55.470761,62.203132,21.582238,405.961084,41.215124
min,-186.280270,-11.560507,-224.896060,-223.307220,-93.272020,-184.962450,-791.469482,-278.582820,94.831150,-142.156950,-330.570600,-352.405200,-214.742460,-145.860280,8.818390,-82.645004,-91.657020,-247.589280,-326.648740,-82.637184,-228.808260,136.873137,-211.971800,-87.323780,-122.584274,-61.803358,-338.562500,-6.629074,-285.668700,111.387250,-130.444920,-290.124000,-25.550720,-1044.135662,-119.903810,-11.926712,-124.560380,-193.918150,-166.078460,-123.062420,-43.063824,-18.980648,-324.943570,-67.540375,-63.102401,-64.789690,-327.322720,-365.060880,-129.879180,-280.371830,-250.337570,-236.148470,18.477040,-354.915700,66.068665,18.104557,-161.189320,-183.849880,-330.638180,-681.029305,-358.460450,-169.566400,-210.059310,-60.779335,-317.831670,-157.590189,60.668890,-203.746380,-181.973820,22.598862,-1297.871984,-209.935760
25%,-103.309147,4.313784,-69.494487,-160.028520,79.474322,-92.834080,-738.536868,-22.131764,118.674642,82.545028,-168.599020,-103.048815,-69.801705,-18.815189,126.369124,-32.379752,-16.455603,-49.612492,-171.661355,-79.656816,-115.407545,1248.126198,-37.047511,31.436791,-5.051067,813.770071,-106.860569,13.187895,-119.613590,145.143008,-32.812503,-168.535163,70.432237,-803.390708,9.505674,59.978086,-30.926924,-153.339210,-127.603408,

In [7]:
# Набор товаров 'base'
data_overview(base)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,-124.06151,-32.458237,-57.420560,36.207405,182.71677,-28.071688,-5.209374,-1.732182,-91.09186,-79.46667,-103.939090,1507.231274,-32.591667,51.414360,-51.905360,813.770071,-41.556538,8.419807,-129.976640,148.76503,-10.069234,-143.00504,79.848694,-537.183707,2.773180,111.518680,4.257666,-153.34058,-123.62608,-39.707664,-12.768708,-4.170106,-152.967130,75.911520,-57.470088,17.266476,-98.871155,-138.06754,91.000380,-170.702000,-50.020390,59.794296,78.106380,-89.375725,89.412100,79.812180,-129.51361,-44.659496,-80.44221,5.149714,-64.518950,-138.77763,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,-141.30435,-126.768900,-68.853530,8.314717,143.08174,8.778257,15.157185,48.246360,-244.95486,-80.27942,-81.788040,429.880035,-5.937690,70.142654,66.576840,813.770071,3.214371,17.164303,-39.418385,148.15588,-7.579590,-174.36995,101.224630,-303.939233,16.914630,53.394676,33.451824,-148.55263,-110.34017,-33.277206,-8.688703,13.650302,-91.228745,38.445015,-31.948470,62.484820,-166.473360,-189.01042,-56.731556,-30.027319,59.171060,-56.711540,63.542606,-121.319170,97.426056,74.832840,-138.77705,58.887608,-71.59534,-251.952358,11.392853,-148.57166,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,-140.82245,-22.486748,-51.666990,2.521726,140.58545,-22.039230,-7.128634,-5.184787,-111.71212,-79.02927,-82.452576,1507.231274,-0.421011,84.108010,63.344510,813.770071,-105.211050,18.844700,-94.789474,151.38771,-21.483440,-144.84537,48.502934,-780.449185,64.767310,79.781555,-13.390142,-150.06166,-118.75855,-40.896286,-15.222620,-1.860338,-93.820656,98.749070,-46.745782,65.927475,-10.286392,-125.32137,21.048609,-22.637775,68.915985,-54.902260,66.335470,-142.927920,83.030980,69.979630,-123.73389,65.320880,-114.03718,-529.396956,-101.581375,-134.65822,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,-108.47044,-104.537780,-82.388490,-79.986740,113.59631,-38.995880,-4.787360,8.327808,-124.85576,-79.10902,-107.943275,1479.170530,-125.271614,24.656485,-16.921055,813.770071,-182.680270,14.715704,-62.932700,151.20198,10.613454,-134.93434,58.777730,-529.295053,43.665924,92.806305,-38.936657,-145.04530,-124.29577,-36.876730,-17.742870,10.536242,-124.584150,121.914150,-17.321358,105.217240,-94.379650,-63.76927,29.217487,-26.423973,5.869829,64.061550,63.513280,-112.258100,80.926590,72.636100,-137.31432,7.912551,-62.18892,-540.321044,-89.588715,-149.32669,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,-110.34245,-38.982060,-50.228867,-26.690920,135.27290,-7.510103,8.924209,111.576950,-130.83319,-79.31585,-57.843952,1262.919738,19.225910,59.811813,76.159670,312.698903,-76.726974,21.299923,-24.069107,167.38553,-44.059470,-148.13185,98.042610,-657.524388,96.363170,71.061520,-36.563557,-135.37244,-123.70898,-44.739437,-10.855220,8.354800,-168.328860,215.928070,-32.975979,66.864880,30.838630,-211.54037,57.695343,-67.822630,-29.389133,-34.123740,54.710550,-26.032013,90.673910,58.086998,-134.78206,77.62

Размер датасета: (2918139, 72)
В датасете нет пропусков


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,-8.622947e+01,8.080077e+00,-4.458080e+01,-1.466350e+02,1.113166e+02,-7.199138e+01,-3.922239e+02,2.035283e+01,1.236842e+02,1.244581e+02,-1.431396e+02,-6.988662e+01,-4.126493e+01,7.587780e+00,1.489852e+02,-2.316701e+01,-9.366916e-01,-1.931312e+01,-1.372712e+02,-7.918932e+01,-9.825383e+01,1.257528e+03,-6.824936e+00,5.667382e+01,2.669563e+01,7.271026e+02,-6.625340e+01,1.631537e+01,-8.637482e+01,1.517346e+02,-1.160764e+01,-1.442328e+02,8.721997e+01,-5.541608e+02,3.623933e+01,7.519918e+01,-1.648670e+01,-1.466421e+02,-1.185122e+02,-4.187896e+01,-9.482269e+00,4.050654e+00,-1.127440e+02,1.208653e+02,-3.980939e+01,5.893878e+01,-8.488885e+01,-1.384695e+02,4.792759e+01,-3.238229e+01,2.658408e+01,-2.171709e+01,6.699684e+01,-1.178976e+02,8.802818e+01,6.920454e+01,-1.329056e+02,1.322645e+01,-1.195996e+02,-3.160935e+02,-6.494332e+01,-1.306677e+02,-7.902286e+01,3.329735e+01,-1.547962e+02,1.415132e+01,6.779167e+01,2.354490e+01,7.495930e+01,1.155667e+02,-7.993390e+02,-4.779125e+01
std,2.489132e+01,4.953387e+00,3.863166e+01,1.984480e+01,4.634809e+01,2.818607e+01,2.716550e+02,6.421638e+01,6.356109e+00,6.443058e+01,4.168737e+01,5.121991e+01,3.896358e+01,3.659725e+01,3.363559e+01,1.307563e+01,2.083201e+01,5.297210e+01,4.662566e+01,7.574403e-01,2.815861e+01,3.999612e+02,5.197251e+01,3.436061e+01,3.894157e+01,2.055724e+02,6.153334e+01,4.656628e+00,4.851313e+01,1.009000e+01,2.895059e+01,3.475739e+01,2.785556e+01,2.606486e+02,3.989506e+01,2.144713e+01,2.585462e+01,9.953913e+00,1.314411e+01,1.919689e+01,8.190720e+00,5.835750e+00,5.694050e+01,4.671995e+01,1.474482e+01,3.211030e+01,6.319335e+01,5.773822e+01,4.520157e+01,5.828693e+01,5.895291e+01,5.173898e+01,1.307173e+01,5.470584e+01,4.774309e+00,1.194907e+01,7.815245e+00,5.107988e+01,5.524317e+01,2.106644e+02,6.248236e+01,9.569063e+00,3.045642e+01,2.888603e+01,4.122929e+01,9.895115e+01,1.823356e+00,5.534224e+01,6.134500e+01,2.117518e+01,3.854131e+02,4.174802e+01
min,-1.994687e+02,-1.391461e+01,-2.400734e+02,-2.326671e+02,-1.055830e+02,-2.110086e+02,-7.914699e+02,-3.018597e+02,9.315305e+01,-1.738719e+02,-3.438436e+02,-3.684526e+02,-2.260906e+02,-1.756239e+02,-1.526698e+01,-8.591711e+01,-9.636816e+01,-2.725656e+02,-3.517478e+02,-8.315235e+01,-2.437279e+02,1.368182e+02,-2.382874e+02,-8.818329e+01,-1.431833e+02,-6.198046e+01,-3.585515e+02,-8.364693e+00,-3.140082e+02,1.074433e+02,-1.567305e+02,-2.986792e+02,-3.465488e+01,-1.045312e+03,-1.378509e+02,-2.151936e+01,-1.379866e+02,-1.954172e+02,-1.769110e+02,-1.327073e+02,-4.757769e+01,-2.158085e+01,-3.381192e+02,-8.770834e+01,-6.310310e+01,-7.724959e+01,-3.629306e+02,-4.186953e+02,-1.453876e+02,-3.228121e+02,-2.802731e+02,-2.646334e+02,6.196133e+00,-3.812643e+02,6.558115e+01,1.408166e+01,-1.670627e+02,-1.961836e+02,-3.597054e+02,-6.810424e+02,-4.007911e+02,-1.713491e+02,-2.205662e+02,-8.850774e+01,-3.539028e+02,-1.575944e+02,5.95094

In [8]:
# Товары, для которых ищем похожие 'validation'
data_overview(validation)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,-131.89280,-140.968570,-57.987164,-22.868887,150.89552,7.965574,17.622066,-34.868217,-216.13855,-80.908730,-52.579520,263.363136,56.266876,66.924710,21.609911,813.770071,-32.782940,20.794031,-79.779076,156.30708,-42.831330,-71.723335,83.283660,-304.174382,1.609402,55.834587,-29.474255,-139.16277,-126.038350,-62.643830,-5.012346,11.984920,-43.084946,190.124000,-24.996636,76.153900,-245.26157,-143.65648,-4.259628,-46.664196,-27.085403,-34.346962,75.530106,-47.171707,92.697320,60.475630,-127.48687,-39.484753,-124.384575,-307.949760,45.506813,-144.19095,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,-153.71725,-63.951330,-52.369495,-33.390945,148.61950,-22.483830,15.164185,-56.202000,-153.61438,-79.831825,-101.055480,1203.537156,81.597130,101.018654,56.783424,92.209628,-126.860340,10.382887,-38.523360,165.38391,-77.840485,-169.538680,103.483240,-915.735701,16.109938,14.669937,-38.707085,-149.53838,-138.792920,-36.076176,-2.781422,2.283144,-142.477890,189.953950,-18.408230,90.517050,-95.53100,-259.63605,52.437836,-30.004599,14.502060,-1.071201,66.842670,-161.279890,94.794174,50.419983,-125.07526,-25.169033,-176.176880,-655.836897,-99.238370,-141.53522,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,-220.30954,-31.378445,-8.788761,2.285323,133.26611,-41.309080,14.305538,-18.231812,-205.53370,-78.160310,-96.607670,1507.231274,-5.964200,34.937443,-56.086887,813.770071,-13.200474,18.966661,-35.110190,151.36850,-17.490252,-145.884300,15.533379,-655.395514,39.412827,62.554955,9.924992,-143.93462,-123.107796,-37.032475,-13.501337,12.913328,-116.038020,176.276150,-45.909942,103.491360,-90.65699,-162.61570,117.128235,13.079479,69.826890,-6.874451,63.707214,-123.851070,91.610820,59.760067,-129.56618,-12.822194,-154.197650,-407.199067,5.522629,-126.81297,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,-114.27196,-71.406456,-65.349320,24.377070,50.46730,-14.721335,15.069309,-46.682995,-176.60437,-78.690700,-139.227450,325.547112,3.632292,74.929504,-4.802103,813.770071,-52.982597,15.644381,-54.087467,151.30914,21.088570,-134.507890,65.118960,-529.295053,131.565520,67.642700,-22.884491,-145.90652,-86.917330,-11.863579,-22.188885,0.463720,-212.533750,170.522580,-48.092532,99.712555,-194.69241,-141.52318,60.217050,73.386380,118.567856,58.900810,55.569030,-181.091660,83.340485,66.083240,-114.04887,-57.156870,-56.335075,-318.680065,-15.984783,-128.10133,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,-76.21417,-26.393860,34.423640,50.938890,157.68318,-23.786497,-33.175415,-0.592607,-193.31854,-79.651030,-91.889786,1358.481072,44.027733,121.527210,46.183000,433.623103,-82.233200,21.068508,-32.940117,149.26895,0.404718,-97.674530,81.719990,-825.644804,9.397169,49.359340,17.725466,-160.16815,-129.367950,-55.532898,-2.597821,-0.226103,-41.369140,92.090195,-58.626857,73.655440,-10.25737,-175.65678,25.395056,47.874825,51.464676,140.951680,58.751133,-215.487640,91.255

Размер датасета: (100000, 72)
В датасете нет пропусков


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.302233,7.669724,-43.842474,-146.119797,111.635071,-73.273042,-441.396759,21.594242,123.077725,126.001062,-141.215384,-69.340237,-42.626768,6.272022,149.006081,-23.961728,-2.009594,-13.151269,-138.540600,-79.162680,-97.356361,1301.395163,-3.439855,56.097825,23.357262,741.156981,-64.986388,16.367853,-86.139332,152.078754,-13.476064,-144.568331,89.709031,-588.597042,36.554961,73.085400,-13.889099,-146.461500,-118.391474,-40.992214,-9.193917,4.157828,-104.775965,122.095524,-40.311826,59.123753,-83.620997,-141.003063,46.187760,-34.154328,29.404870,-22.542637,67.486142,-120.307717,88.174847,69.495540,-132.377437,14.411927,-120.381837,-316.401557,-66.614887,-130.653256,-81.129311,36.778882,-152.341249,14.397762,67.788892,23.250779,73.114446,115.196935,-709.457021,-48.416276
std,25.777321,4.956990,39.138775,20.495541,47.751576,28.515740,279.242585,66.478048,6.504931,64.981697,41.568432,51.483403,40.858017,37.845939,34.031665,13.014670,20.993654,55.340411,48.790143,0.777431,28.503558,371.543911,50.291410,35.599512,40.849607,193.199671,63.274623,4.845841,49.511970,10.406558,29.275776,35.911251,28.430405,273.788805,39.610961,20.044353,25.802824,10.389778,13.602543,19.845968,8.364438,5.910225,56.986969,48.130993,14.753263,31.126990,65.925312,59.595488,45.996789,59.611850,60.943297,55.140398,13.036243,56.107653,4.809309,12.324416,7.894675,49.387024,55.818220,211.131373,65.195278,9.426921,30.691590,25.427103,41.246347,98.695231,1.844522,55.403862,62.056224,21.493081,405.665764,41.292843
min,-190.353330,-11.109877,-217.538420,-220.050890,-81.198990,-176.705030,-791.460243,-265.608430,96.218350,-135.667300,-321.544250,-363.493320,-204.299130,-144.100740,-14.816055,-78.597450,-86.843280,-272.565600,-329.860170,-82.665300,-231.376650,136.846377,-208.900850,-84.224530,-138.217000,-61.879841,-341.962220,-6.662937,-282.249330,110.357254,-150.154720,-281.123540,-24.076744,-1044.135662,-117.473145,-17.641396,-124.311615,-194.132930,-166.124450,-126.336075,-43.321170,-19.319794,-326.301300,-79.304720,-63.102401,-76.792040,-365.282530,-375.452200,-121.267914,-277.375920,-279.886380,-234.521500,12.404751,-381.609380,65.986030,17.301888,-162.872990,-156.581590,-329.700800,-681.038139,-339.966340,-168.735570,-215.152280,-66.265970,-337.632870,-157.593541,60.377728,-210.672800,-175.921780,25.271042,-1297.923999,-209.935760
25%,-103.231790,4.292425,-69.680916,-160.210287,79.101920,-92.789803,-740.620025,-21.804730,118.657689,82.311205,-168.562772,-103.338022,-70.147153,-19.089412,126.442184,-32.356516,-16.392179,-49.942476,-171.695623,-79.665475,-115.390365,1248.126198,-36.891637,31.741733,-5.265194,813.770071,-107.339855,13.182383,-119.527813,145.126130,-32.815898,-168.729353,70.166879,-803.390708,9.700724,59.971492,-30.786210,-153.332742,-127.66153

Можно заметить, что данные разделены на обучающую и тестовую выборки по порядку, ориентируясь на индексы

In [9]:
# Ответы к предыдущему набору товаров из 'validation' - 'answers'
data_overview(answers)

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


Размер датасета: (100000, 1)
В датасете нет пропусков


,Expected
count,100000
unique,91502
top,210304-base
freq,7


В данном разделе проекта мы:
- загрузили данные и вывели информацию о них
- определили, что в данных отсутствуют пропуски
- определили обучающие и валидационные выборки (набор признаков и таргет)  


## Подготовка данных к построению моделей

Обучающий датасет содержит набор признаков и таргет, которые нужно разделить

In [10]:
target = train['Target']
features = train.drop('Target', axis=1)

Переименуем признаки и таргет из валидационного сета

In [11]:
features_test = validation
target_test = answers['Expected']

Разделим обучающий датасет на тренировочную и валидационную выборки

In [12]:
# Сократим обучающую выборку до 50000 товаров
features_train, features_extra, target_train, target_extra = (
    train_test_split(features, target, test_size=0.9, random_state=12345)
)

Так как разброс в значениях данных большой, проведем масштабирование признаков

In [13]:
# Проведем масштабирование признаков

numeric = features_train.columns.to_list() # Запишем все количественные признаки в список numeric
# numeric

features_train_scaled = features_train.copy()
features_test_scaled = features_test.copy()
base_scaled = base.copy()

# Используем метод StandardScaler
# Он приводит все значения к стандартному нормальному распределению
scaler = StandardScaler()
scaler.fit(features_train_scaled[numeric])
features_train_scaled[numeric] = scaler.transform(features_train_scaled[numeric])
features_test_scaled[numeric] = scaler.transform(features_test_scaled[numeric])
base_scaled[numeric] = scaler.transform(base_scaled[numeric])

#features_train_scaled.head()
#features_valid_scaled.head()
#features_test_scaled.head()

In [16]:
# Удалим ненужные переменные
del (
    answers, base, base_url, delete_filepath, download_url, f,
    features, target, features_train, features_test, final_url,
    numeric, public_key, response, scaler, train, validation
)

В данном разделе мы:
- сохранили обучающую и тестовую выборки, отделив признаки от таргета
- провели масштабирование признаков  
  
Данные готовы к построению моделей

## Построение моделей

### FAISS

Построим модель FAISS с ускоренным поиском с помощью inverted lists. Посчитаем предсказания в два этапа.  
  
**Первый этап:**
- Сначала сгруппируем набор векторов из датасета `base` по частям с помощью k-means
- Для каждой части найдем центроид
- Далее будем искать для каждого вектора из `features_valid_scaled` наиболее близкий центроид
- В части с наиболее близким к вектору центроидом будем искать наиболее близкий вектор  
  
*Результатом первого этапа будет модель Faiss для предсказания 30 наиболее близких векторов к каждому из обучающей выборки*
  
**Второй этап:**
- Объединим полученыые предсказания и изначальные вектора, так у нас получится датасет с 72 * 2 = 144 признаками
- Добавим в признаки расстояние от каждого вектора из обучающей выборки до ближайших
- Посчитаем новый таргет, где 1 - предсказание модели соответствует рекомендации эксперта, 0 - нет
- Построим CatBoost, предсказывающий вероятность выбора конкретного товара в качестве наиболее похожего

#### Первый этап (FAISS)

Создадим индекс

In [17]:
n_cells = round(base_scaled.shape[0] ** 0.5) # количество кластеров
print(f'Разделим датасет на {n_cells} кластеров при создании индекса')

Разделим датасет на 1708 кластеров при создании индекса


Обучим индекс и добавим в него вектора из `base_scaled` для получения предсказаний

In [18]:
dim = features_train_scaled.shape[1] # размерность вектора признака
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantiser, dim, n_cells) # создание индекса

In [19]:
index.train(np.ascontiguousarray(features_train_scaled.values).astype('float32')) # обучим индекс на трейне
print(index.is_trained) # проверим обучение

index.add(np.ascontiguousarray(base_scaled.values).astype('float32')) # добавим вектора в индекс

True


In [20]:
# Пронумеруем id товаров, запишем в словарь
base_index = {n: v for n, v in enumerate(base_scaled.index.to_list())}
# base_index

Посчитаем предсказания на обучающей выборке и оценим качество модели с гиперпараметром nprobe = 300. Так как нам нужно будет использовать код повторно для оценки качества модели на тестовой выборке, напишем функцию для вычисления предсказаний

In [21]:
'''
Функция принимает на вход заскейленные признаки и таргет
Функция возвращает просчитанные индексы ближайших векторов и расстояния до них, а также метрику accuracy@30 и время подсчета
'''

def faiss_target_formation(features_faiss, target_faiss):

    start = time.time()
    index.nprobe = 300 # обозначим количество кластеров для поиска соседей
    vecs_new, idx_new = index.search(np.ascontiguousarray(features_faiss.values).astype('float32'), 30) # найдем вектора на трейне
    end = time.time()

    acc = 0
    for t, el in zip(target_faiss.values.tolist(), idx_new.tolist()): # посчитаем accuracy@30 для модели с nprobe=n
        acc += int(t in [base_index[r] for r in el])

    acc_faiss = 100 * acc / len(idx_new)

    faiss_time = end - start

    return vecs_new, idx_new, faiss_time, acc_faiss

In [ ]:
'''
vecs, idx, faiss_time, acc_faiss = faiss_target_formation(features_train_scaled, target_train)
# print(acc_faiss, faiss_time)
'''

In [22]:
acc_faiss = 74.18
faiss_time = 507.5104320049286

print(f'Точность алгоритма faiss на nprobe=100 и 1708 кластерах: {acc_faiss}\n'
    f'Время подбора ближайших векторов для трейна модели faiss: {faiss_time}')

Точность алгоритма faiss на nprobe=100 и 1708 кластерах: 74.18
Время подбора ближайших векторов для трейна модели faiss: 507.5104320049286


In [ ]:
'''
# Скачаем на диск полученные индексы для train
with open('/content/drive/My Drive/idx.txt', 'w') as f:
    np.savetxt(f, idx)

# Скачаем на диск расстояния до ближайших векторов для каждого товара для train
with open('/content/drive/My Drive/vecs.txt', 'w') as f:
    np.savetxt(f, vecs)
'''

In [23]:
with open('/content/drive/My Drive/idx.txt', 'r') as f:
    idx = np.loadtxt(f)

with open('/content/drive/My Drive/vecs.txt', 'r') as f:
    vecs = np.loadtxt(f)

Метрика accuracy@n получилась 74.18 при nprobe=100. Попробуем улучшить результат на втором шаге, используя CatBoost

#### Второй этап (CatBoost)

Напишем функцию для формирования признаков и таргета для CatBoost

In [24]:
'''
Функция принимает на вход признаки и таргет faiss
Функция возвращает склеенные признаки старых векторов и ближайших к ним, таргет для катбуста (1-вектор правильно побобран моделью, 0-неправильно)
'''

def catboost_features_formation(features_faiss, target_faiss, idx_faiss, vecs_faiss):

    rows = np.repeat(features_faiss.values, 30, axis=0).shape[0]
    cols = np.repeat(features_faiss.values, 30, axis=0).shape[1]

    added_features = np.zeros(shape=(rows, cols)) # Создадим датасет размерностью 100,000 * 50 на 72 для новых признаков
    idx_reshaped = idx.reshape(rows) # запишем индекс в более удобной форме
    vecs_faiss = vecs_faiss.reshape(-1, 1)

    for row in range(rows):
         added_features[row] = base_scaled.loc[base_index[idx_reshaped[row]]]

    features_catboost = np.concatenate(
        (np.repeat(features_faiss.values, 30, axis=0), added_features),
            axis=1
    )

    features_catboost = np.concatenate(
        (features_catboost, vecs_faiss),
            axis=1
    )

    target_catboost = []
    for t, el in zip(target_faiss.values.tolist(), idx_faiss.tolist()):
        for r in el:
             if base_index[r] == t:
                 target_catboost.append(1)
             else:
                 target_catboost.append(0)

    target_catboost = np.array(target_catboost)

    # target_catboost.sum() == acc

    return features_catboost, target_catboost

In [25]:
features_catboost, target_catboost = catboost_features_formation(features_train_scaled, target_train, idx, vecs)
# 100 * target_catboost.sum() / len(idx) == acc_faiss

Подберем гиперпараметры для CatBoost (второго этапа метчинга)

In [ ]:
'''
start = time.time()
grid_space_catboost = {
    'iterations': [550],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': np.arange(5, 8)
}

model_catboost = CatBoostClassifier(random_state=12345, verbose=False,
                                    auto_class_weights='Balanced',
                                    eval_metric='Accuracy')

grid_search_result = model_catboost.grid_search(grid_space_catboost, X=features_catboost,
                                                y=target_catboost, cv=3, verbose=False)
end = time.time()
'''


bestTest = 0.8489783634
bestIteration = 549


bestTest = 0.9057586594
bestIteration = 521


bestTest = 0.9055535656
bestIteration = 290


bestTest = 0.8580301076
bestIteration = 548


bestTest = 0.8998896501
bestIteration = 444


bestTest = 0.9028305796
bestIteration = 175


bestTest = 0.8685708152
bestIteration = 548


bestTest = 0.8984212011
bestIteration = 251


bestTest = 0.8994442512
bestIteration = 138

Training on fold [0/3]

bestTest = 0.9008142136
bestIteration = 329

Training on fold [1/3]

bestTest = 0.910775199
bestIteration = 528

Training on fold [2/3]

bestTest = 0.9005224652
bestIteration = 364



In [26]:
# best_params = grid_search_result['params']
best_params = {'depth': 5, 'iterations': 550, 'learning_rate': 0.1}
print(f'Лучшие гиперпараметры модели CatBoost: {best_params}')

# catboost_time = end - start
catboost_time = 1168.5561311244965
print(f'Впемя обучения модели CatBoost: {catboost_time}')

Лучшие гиперпараметры модели CatBoost: {'depth': 5, 'iterations': 550, 'learning_rate': 0.1}
Впемя обучения модели CatBoost: 1168.5561311244965


In [27]:
model_catboost = CatBoostClassifier(random_state=12345, verbose=False,
                                    depth=5, iterations=550,
                                    learning_rate=0.1, auto_class_weights='Balanced',
                                    eval_metric='Accuracy')

model_catboost.fit(features_catboost, target_catboost)

predictions_train = model_catboost.predict_proba(features_catboost)[:, 1]

Основываясь на accuracy score на кросс-валидации, качество модели улучшилось после добавления CatBoost. Посчитаем accuracy@5 на обучающей выборке, чтобы в этом убедиться. Напишем функцию для ее подсчета

In [28]:
def accuracy_5(predictions, target):

    # Выведем индексы номеров, на которых стоят 5 наиболее вероятных товаров для каждого из обучающей выорки
    predictions = pd.DataFrame(predictions.reshape(-1, 30))
    listed_idx = []
    for row in range(predictions.shape[0]):
        listed_idx.append(predictions.loc[row, :].sort_values(ascending=False)[:5].index.to_list())


    # Поставим 1 на место пяти наиболее вероятных товаров, предсказанных катбустом
    predictions = np.zeros(shape=predictions.shape)
    for row in range(predictions.shape[0]):
        for elem in listed_idx[row]:
            predictions[row][elem] = 1

    # Посчитаем accuracy@5
    target = target.reshape(-1,)
    predictions = predictions.reshape(-1,)
    acc = len([i for i, j in zip(target, predictions) if i == j and i == 1]) / target.reshape(-1, 30).shape[0]

    return acc

In [29]:
accuracy_5(predictions_train, target_catboost) * 100

73.65

Accuracy@5 после катбуста немного упала. Нужно посчитать метрику на тестовой выборке, чтобы убедиться, что второй этап с градиентным бустингом нужен

In [30]:
del idx, vecs, features_catboost, target_catboost

## Проверка модели на тестовой выборке

In [31]:
# Сократим тестовую выборку до 50000 товаров
features_test, features_extra_test, target_test, target_test = (
    train_test_split(features_test_scaled, target_test, test_size=0.9, random_state=12345)
)

In [46]:
'''
# Посчитаем ближайшие векторы и расстояния для сокращенной тестовой выборки
vecs_test, idx_test, faiss_time, acc_faiss = faiss_target_formation(features_test, target_test)
'''

In [26]:
'''
# Скачаем на диск полученные индексы для train
with open('/content/drive/My Drive/vecs_test.txt', 'w') as f:
    np.savetxt(f, vecs_test)

# Скачаем на диск расстояния до ближайших векторов для каждого товара для train
with open('/content/drive/My Drive/idx_test.txt', 'w') as f:
    np.savetxt(f, idx_test)
'''

In [32]:
with open('/content/drive/My Drive/idx_test.txt', 'r') as f:
    idx = np.loadtxt(f)

with open('/content/drive/My Drive/vecs_test.txt', 'r') as f:
    vecs = np.loadtxt(f)

In [35]:
acc = 0
for t, el in zip(target_test.values.tolist(), idx.tolist()): # посчитаем accuracy@30 для модели с nprobe=n
    acc += int(t in [base_index[r] for r in el])

acc_faiss = 100 * acc / len(idx)
acc_faiss

0.02

Метрика на faiss получилась очень низкая

In [36]:
# Обозначим признаки и таргет для катбуста, найдем предсказания из катбуста
features_catboost_test, target_catboost_test = catboost_features_formation(features_test, target_test, idx, vecs)
predictions_test = model_catboost.predict_proba(features_catboost_test)[:, 1]

In [37]:
accuracy_5(predictions_test, target_catboost_test)

0.0